# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.**

In [ ]:
median_values = data.groupby('income_type')['total_income'].median()
for i in range(len(data)):
     if pd.isnull(data.loc[i, 'total_income']):  
        income_type = data.loc[i, 'income_type']  
        data.loc[i, 'total_income'] = median_values[income_type] 
        

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

**Выведим перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
median_days = data.groupby('income_type')['days_employed'].median()


def fill_missing_days(row):
    if pd.isnull(row['days_employed']):
        income_type = row['income_type']
        if income_type in median_days:
            return median_days[income_type]
        else:
            return None  
    return row['days_employed']

data['days_employed'] = data.apply(fill_missing_days, axis=1)  

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [ ]:
def categorize_income(total_income):
    if 0 < total_income <= 30000:
        return 'E'
    if 30000 < total_income <= 50000:
        return 'D'
    if 50000 < total_income <= 200000:
        return 'C'
    if 200000 < total_income <= 1000000:
        return 'B'
    else:
        return 'A'

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [ ]:
def categorize_purpose(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недвижимо' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    if 'образова' in purpose:
        return 'получение образования'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для проверки первой гипотезы нам потребуются данные о количестве детей у заемщика и информация о том, имел ли он задолженность по возврату кредитов.

In [22]:
#Сгруппируем данные по количеству детей и подсчитаем количесвто должников
children_debt = data.groupby('children')['debt'].agg(['count', 'sum'])
children_debt = children_debt.rename(columns = {'count': 'total_clients', 'sum': 'clients_with_debt'})
#В столбце total_clients показано общее количество заемщиков с определенным количеством детей
#В столбце clients_with_debt - количество заемщиков с долгами среди группы
children_debt

,total_clients,clients_with_debt
children,,
0,14091,1063
1,4808,444
2,2052,194
3,330,27
4,41,4
5,9,0


In [23]:
#Добавим столбец с процентным соотношением
children_debt['ratio'] = children_debt['clients_with_debt'] / children_debt['total_clients']
children_debt

,total_clients,clients_with_debt,ratio
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** 

1. Меньше всего должников среди людей у которых нет детей (7.5%).
2. Сначала с увеличением увеличением количества детей наблюдается рост количества должников, но на третьем ребенке данный показатель уменьшается.
3. Группа с 4 детьми имеет наивысший процент должников (9.7%).
4. Группа с 5 детьми имеет слишком малый размер выборки, с этим связано отсутствия должников в данной группе.

Таким образом, первая гипотеза подтверждается лишь частично: 

* доля должников среди заёмщиков с разным количеством детей отличается. Так, заёмщики с 1-2 или 4 детьми имеют чуть более высокую долю задолженности, чем те, у кого нет детей.
* Группа 4 и 5 имеют маленькую выборку, что делает статистические данные менее надежными.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для проверки второй гипотезы нам потребуются данные о семейном положении заёмщика и информация о том, имел ли он задолженность по возврату кредитов.

In [24]:
#Сгруппируем данные по семейному положению и подсчитаем количесвто должников
family_status_debt = data.groupby('family_status')['debt'].agg(['count', 'sum'])
family_status_debt = family_status_debt.rename(columns = {'count': 'total_clients', 'sum': 'clients_with_debt'})
#Добавим столбец с процентным соотношением
family_status_debt['ratio'] = family_status_debt['clients_with_debt'] / family_status_debt['total_clients']
family_status_debt

,total_clients,clients_with_debt,ratio
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** 

1. Больше всего должников наблюдается среди людей, которые не состаят в официальных отношениях (не женат/не замужем, гражданский брак).
2. Люди, которые находятся в официальном браке или в разводе имеют меньшую долю должников, чем те, которые не не состаят в официальных отношениях.
3. Наименьшая доля должников среди вдов/вдовцов.

Вторая гипотеза подтверждается. Наличие официального или бывшего брачного статуса связано с более низким процентом невозврата кредита. Это может быть связано с большей финансовой отвественностью среди этих групп. 



#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для проверки третьей гипотезы нам потребуются данные об уровне дохода заёмщика и информация о том, имел ли он задолженность по возврату кредитов.

In [25]:
#Сгруппируем данные по уровню дохода и подсчитаем количесвто должников
total_income_debt = data.groupby('total_income_category')['debt'].agg(['count', 'sum'])
total_income_debt = total_income_debt.rename(columns = {'count': 'total_clients', 'sum': 'clients_with_debt'})
total_income_debt['ratio'] = total_income_debt['clients_with_debt'] / total_income_debt['total_clients']
total_income_debt

,total_clients,clients_with_debt,ratio
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** 

1. Количество должников в категории A - 8%, но выборка невилека, что может повлиять на достоверность выводов.
2. Для B - 7.06%, для C - 8.49%. В категории C наблюдается самое большое количество должников после категории E (в этой категории выборка мала).
3. В категории D наблюдается наименьший процент должников - 0.6%.

Таким образом третья гипотеза не подтверждается. Нельзя выделить четкую закономерность влияния доходов на количество должников.


#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Для проверки четвертой гипотезы нам потребуются данные цели кредита и информация о том, имел ли заемщик задолженность по возврату кредитов.

In [26]:
 #Сгруппируем данные по цели кредита и подсчитаем количесвто должников
purpose_category_debt = data.groupby('purpose_category')['debt'].agg(['count', 'sum'])
purpose_category_debt = purpose_category_debt.rename(columns = {'count': 'total_clients', 'sum': 'clients_with_debt'})
purpose_category_debt['ratio'] = purpose_category_debt['clients_with_debt'] / purpose_category_debt['total_clients']
purpose_category_debt

,total_clients,clients_with_debt,ratio
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Вывод:** 

1. Операции с недвижимостью имеют самую низкую долю должников. Это может свидетельствовать о том, что кредиты на недвижимость заемщики чаще возвращают в срок, возможно, потому что они рассматриваются как важные и долгосрочные вложения.
2. Операции с автомобилем, а также получение образования имеют наибольший доли должников. Возможно, что кредиты на данные цели сложнее выплатить из-за того, что эти расходы могут быть менее приоритетными.
3. Кредиты на проведение свадеб имеют среднюю долю должников.

Таким образом, четвертая гипотеза подтверждается. Цель кредита влияет на то, будет ли он погашен в срок.

#### 3.5 Приведем возможные причины появления пропусков в исходных данных.

*Ответ:* 

Во время предобработки данных пропуски были обнаружены в столбце days_employed(общий трудовой стаж в днях) и столбце total_income(ежемесячный доход). При этом количество пропусков в этих столбцах было одинаковым. 

Можно выделить следующие причины:

* Во время сбора данных могла произойти ошибка.
* Некоторые люди могли не заполнить эти данные.
* Некоторые люди могли работать не официально.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 

В отличие от среднего значения, которое может некорректно характеризовать данные, когда некоторые значения сильно выделяются среди большинства, медиана - это такое число в выборке, что ровно половина элементов больше него, а другая - меньше. В нашем случае лучше использовать медиану, так как в данных могут быть выдающиеся значения, которые могут сильно сместить среднее.

### Шаг 4: общий вывод.

Было проверено четыре гипотезы. В результате установлено:

1. Доля должников несколько выше среди клиентов с 1-2 или 4 детьми по сравнению с теми, у кого детей нет.

Гипотеза о том, что количество детей влияет на возврат кредита в срок, подтвердилась частично.

2. Существует зависимость между семейным положением и возвратом кредита в срок. Наличие официального или бывшего брачного статуса связано с более низким процентом невозврата кредита. Это может быть связано с большей финансовой ответственностью среди этих групп.

Таким образом, вторая гипотеза подтвердилась.


3. Не удалось выделить четкую зависимость между уровнем дохода и возвратом кредита в срок. 


Третья гипотеза не подтвердилась.


4. Цель кредита влияет на то, будет ли он погашен в срок. Так, операции с недвижимостью имеют минимальный процент должников.

Четвертая гипотеза подтверждается.
